In [6]:
!pip install fpdf

In [7]:
import ee
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from fpdf import FPDF

ee.Authenticate()
ee.Initialize(project='resolute-return-377811')

# Define Norfolk Broads wetland locations
locations = [
    (52.7384, 1.5812),  # Hickling Broad
    (52.6160, 1.4675),  # Strumpshaw Fen
    (52.6765, 1.4682),  # Ranworth Broad
    (52.7590, 1.4864),  # Barton Broad
    (52.7413, 1.6448)   # Horsey Mere
]

# Create GeoDataFrame
points = gpd.GeoDataFrame({'geometry': [Point(lon, lat) for lat, lon in locations]})

# Compute Convex Hull
convex_hull = points.unary_union.convex_hull

# Create 50 km buffer
buffered_hull = convex_hull.buffer(50000 / 111320)  # Approximate conversion from meters to degrees

# Get water bodies layer from GEE
water_layer = ee.FeatureCollection("GLCF/GLS_WATER")

# Convert hull to GEE geometry
hull_geometry = ee.Geometry.Polygon(list(convex_hull.exterior.coords))

# Filter water bodies within the convex hull
water_within_hull = water_layer.filterBounds(hull_geometry)

# Convert to GeoJSON for plotting
geojson = water_within_hull.getInfo()
print(geojson)
water_shapes = [
    Polygon(feature['geometry']['coordinates'][0])
    for feature in geojson['features']
]

# Plot
fig, ax = plt.subplots(figsize=(8, 8))
gpd.GeoSeries([convex_hull]).plot(
    ax=ax, edgecolor='red', facecolor='none', linewidth=2, label='Convex Hull'
)
gpd.GeoSeries([buffered_hull]).plot(ax=ax, edgecolor='blue', facecolor='none', linestyle='dashed', label='50 km Buffer')
gpd.GeoSeries(water_shapes).plot(ax=ax, color='cyan', alpha=0.5, label='Water Bodies')
points.plot(ax=ax, color='black', marker='o', label='Wetland Locations')
ax.legend()
ax.set_title("Wetland Analysis - Norfolk Broads")
plt.savefig("wetland_map.pdf")
plt.show()

# Create PDF report
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, "Wetland Analysis - Norfolk Broads", ln=True, align='C')
pdf.image("wetland_map.pdf", x=10, y=30, w=180)
pdf.output("wetland_report.pdf")

print("PDF report generated: wetland_report.pdf")


<ipython-input-7-90fc0837a3b3>:23: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  convex_hull = points.unary_union.convex_hull


KeyError: 'geometry'